In [49]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize
from sklearn.model_selection import train_test_split
from gensim import corpora, models, similarities
from gensim.corpora import Dictionary
from math import sqrt

In [50]:
def make_category_1(file_path):


    f = open(file_path).read()
    data = json.loads(f)
    playlists = data["playlists"]
    df = pd.DataFrame(playlists)
    train, test = train_test_split(df, test_size=0.1)

    r_test_data = {'pid' : list(test['pid']), 'name' : list(test['name']), 'num_holdouts' : list(test['num_tracks']), 'tracks' : [{} for _ in range(test.shape[0])], 'num_samples' : [0 for _ in range(test.shape[0])], 'num_tracks' : test['num_tracks']}
    r_test = pd.DataFrame(r_test_data)
    
    train = train.T.to_dict().values()
    test = test.T.to_dict().values()
    r_test = r_test.T.to_dict().values()
    
    with open('./train.json', 'w+') as f:
        f.write(json.dumps(train))
    with open('./test_R.json', 'w+') as f:
        f.write(json.dumps(test))
    with open('./test.json', 'w+') as f:
        f.write(json.dumps(r_test))
        
        
make_category_1('mpd.slice.0-999.json')

<h1> Train_set</h1>

In [64]:
f = open('./train.json').read()
data = json.loads(f)
train_df = pd.DataFrame(data)
train_df

,collaborative,description,duration_ms,modified_at,name,num_albums,num_artists,num_edits,num_followers,num_tracks,pid,tracks
0,false,NaN,11532414,1493424000,Throwbacks,47,37,6,1,52,0,"[{u'pos': 0, u'artist_name': u'Missy Elliott',..."
1,false,NaN,11656470,1506556800,Awesome Playlist,23,21,5,1,39,1,"[{u'pos': 0, u'artist_name': u'Survivor', u'tr..."
2,false,NaN,28926058,1501027200,mat,107,86,4,1,126,3,"[{u'pos': 0, u'artist_name': u'Camille Saint-S..."
3,false,NaN,4335282,1401667200,90s,16,16,7,2,17,4,"[{u'pos': 0, u'artist_name': u'The Smashing Pu..."
4,false,NaN,3408479,1477094400,I Put A Spell On You,15,13,2,1,16,6,"[{u'pos': 0, u'artist_name': u'Creedence Clear..."
5,false,NaN,4297488,1501804800,old country,20,18,10,1,21,9,"[{u'pos': 0, u'artist_name': u'Willie Nelson',..."
6,false,NaN,16403398,1509321600,abby,60,40,36,2,72,10,"[{u'pos': 0, u'artist_name': u'Post Malone', u..."
7,false,NaN,35655891,1365379200,VIBE,117,99,16,1,134,11,"[{u'pos': 0, u'artist_name': u'Natural Self', ..."
8,false,NaN,1981349,1481587200,relax,9,9,3,1,9,12,"[{u'pos': 0, u'artist_name': u'Yael Naim', u't..."
9,false,NaN,25837498,1382572800,90's,79,61,3,1,103,14,"[{u'pos': 0, u'artist_name': u'311', u'track_u..."


<h1> Test_set</h1>

In [52]:
f = open('./test_R.json').read()
data = json.loads(f)
test_df = pd.DataFrame(data)

<h1> cross validation </h1>

In [62]:
train_playlist_name = list(set(train_df['name']))

train_playlist_name =  [train_playlist_name[i:i + len(train_playlist_name)/10] 
                       for i in xrange(0, len(train_playlist_name), len(train_playlist_name)/10)]
topic_num = 3
max= 0
max_topic_num = 0
for idx in range(0, len(train_playlist_name)):
    #0~9 까지 validation
    validation_texts = [[word for word in name.lower().split()]for name in train_playlist_name[idx]]
    
    #train_set after slice
    train_texts = []
    for tidx in range(0,len(train_playlist_name)):
        if(tidx == idx):
            continue
        train_texts.extend(train_playlist_name[tidx])
    train_texts = [[word for word in name.lower().split()]
                  for name in train_texts]
    
    #LDA model create
    dictionary = corpora.Dictionary(train_texts)
    corpus = [dictionary.doc2bow(text) for text in train_texts]
    model = models.ldamodel.LdaModel(corpus = corpus, id2word = dictionary,
                                    num_topics = topic_num, random_state = 0,
                                    chunksize = 2000, passes = 10)
    
    #document 별 topic 다항분포 train
    train_dic = {}
    for idx in range(0, len(train_texts)):
        t = train_texts[idx]
        bow = dictionary.doc2bow(t)
        get_document_topics = tuple(model.get_document_topics(bow))
        name = ' '.join(t)
        train_dic[name] = dict(get_document_topics)
    train_dic = pd.DataFrame(train_dic)
    #print train_dic
    
    #document 별 topic 다항분포 validation
    validation_dic = {}
    for idx in range(0, len(validation_texts)):
        t = validation_texts[idx]
        bow = dictionary.doc2bow(t)
        get_document_topics = tuple(model.get_document_topics(bow))
        name = ' '.join(t)
        validation_dic[name] = dict(get_document_topics)
    validation_dic =pd.DataFrame(validation_dic).T
    #print validation_dic
    
    result = []
    for trainName in train_dic.columns:
        idx = 0
        c = 0
        columns_ones = []
        for valName in validation_dic.index:
            for idx in range(0, topic_num):
                c += pow(train_dic[trainName][idx] - validation_dic[idx][valName], 2)
            c = 1/(1+sqrt(c))
            columns_ones.append(c)
        result.append(columns_ones)
    #train_playlist X validation_playlist_name 유클리디안 거리 행렬 생성    
    result = pd.DataFrame(result, columns = validation_dic.index, index = train_dic.columns)
    
    #validation_Set evaluation
    count = 0
    precision = 0
    recall = 0
    F1Score = 0
    for name in result.columns:
        #test_playlist_name 과 거리가 가까운 train_playlist_name 상위 10개 도출
        result_playlist_name = result[name].sort_values(ascending = False).head(10).index
        #print count
        count+=1
    
        #test_playlist 생성
        validation_playlist = []
        for idx in range(0, len(train_df['name'])):
            if(train_df['name'][idx].lower() == name):
                val_track_uri = pd.DataFrame(train_df['tracks'][idx])
                val_playlist = list(val_track_uri['track_uri'])
        #print len(test_playlist)        
        if(len(val_playlist) == 0):
            continue
            
        #비교할 train_playlist 생성
        train_playlist = []
        for train_name in result_playlist_name:
            for idx in range(0,len(train_df['name'])):
                if(train_df['name'][idx].lower() == train_name):
                    df = pd.DataFrame(train_df['tracks'][idx])
                    train_playlist += list(df['track_uri'])
                    #if(len(train_playlist) >= 500):
                     #   continue
                    #elif(len(train_playlist) + len(list(df['track_uri'])) >= 500):
                     #   temp = list(df['track_uri'])
                     #   train_playlist += temp[:-(len(train_playlist)-500)]
                    #else:
                        #train_playlist += list(df['track_uri'])
        #print len(train_playlist)      
        #r-precision 계산
        precision += 1.0 *len(set(train_playlist).intersection(set(val_playlist)))/len(set(train_playlist)) *100
        recall += 1.0 * len(set(train_playlist).
                            intersection(set(val_playlist)))/len(set(val_playlist))*100
    #avarage result
    precision = 1.0 * precision/(count)
    recall = 1.0 * recall/(count)
    F1Score = (2 * precision * recall) / (precision + recall)
    print ("precision average : {}\nrecall average : {}\nf1score : {}\ntopic_num : {}\n".format(precision, recall, F1Score, topic_num))
    print
    if max < F1Score:
        max = F1Score
        max_topic_num = topic_num
        maxModel = model

    topic_num += 2 #3,5,7,9,11,13,15,....
print ("max_f1Score : {}, max_topic_num : {}".format(max, max_topic_num)) 

precision average : 0.526738187536
recall average : 7.47260975183
f1score : 0.984107441425
topic_num : 3


precision average : 1.02358443215
recall average : 12.9874245127
f1score : 1.89761145643
topic_num : 5


precision average : 0.685211290235
recall average : 12.0898917633
f1score : 1.29691796602
topic_num : 7


precision average : 0.52491132666
recall average : 10.0054955402
f1score : 0.997491930616
topic_num : 9


precision average : 0.952143003864
recall average : 10.4094876687
f1score : 1.74470041198
topic_num : 11


precision average : 1.06343591218
recall average : 12.5804600454
f1score : 1.96109865476
topic_num : 13


precision average : 0.803821145394
recall average : 12.4097325119
f1score : 1.5098444613
topic_num : 15


precision average : 1.29111213649
recall average : 15.2818629316
f1score : 2.38105694579
topic_num : 17


precision average : 1.01774183926
recall average : 11.879639451
f1score : 1.87486216504
topic_num : 19


precision average : 1.51014970959
recall avera

In [63]:
train_playlist_name = list(set(train_df['name']))
test_playlist_name = list(set(test_df['name']))

#train_playlist_name
#소문자로
train_texts = [[word for word in name.lower().split()]
         for name in train_playlist_name]

#test_playlist_name
#역시 소문자로
test_texts = [[word for word in name.lower().split()]
         for name in test_playlist_name]

topic_num = max_topic_num
 #LDA model create
#dictionary = corpora.Dictionary(train_texts)
#corpus = [dictionary.doc2bow(text) for text in train_texts]
#model = models.ldamodel.LdaModel(corpus = corpus, id2word = dictionary,
#                                   num_topics = topic_num, random_state = 0,
#                                    chunksize = 20000, passes = 10)
#dictionary = corpora.Dictionary(train_texts)
#corpus = [dictionary.doc2bow(text) for text in train_texts]
#print corpus
model = maxModel   
    #document 별 topic 다항분포 train
train_dic = {}
for idx in range(0, len(train_texts)):
    t = train_texts[idx]
    bow = dictionary.doc2bow(t)
    #print bow
    get_document_topics = tuple(model.get_document_topics(bow))
    name = ' '.join(t)
    train_dic[name] = dict(get_document_topics)
train_dic = pd.DataFrame(train_dic)
#print train_dic
    
    #document 별 topic 다항분포 validation
test_dic = {}
for idx in range(0, len(test_texts)):
    t = test_texts[idx]
    bow = dictionary.doc2bow(t)
    get_document_topics = tuple(model.get_document_topics(bow))
    name = ' '.join(t)
    test_dic[name] = dict(get_document_topics)
test_dic =pd.DataFrame(test_dic).T
#print validation_dic
    
result = []
for trainName in train_dic.columns:
    idx = 0
    c = 0
    columns_ones = []
    for testName in test_dic.index:
        for idx in range(0, topic_num):
            c += pow(train_dic[trainName][idx] - test_dic[idx][testName], 2)
        c = 1/(1+sqrt(c))
        columns_ones.append(c)
    result.append(columns_ones)
#train_playlist X validation_playlist_name 유클리디안 거리 행렬 생성    
result = pd.DataFrame(result, columns = test_dic.index, index = train_dic.columns)
#print result   
    #validation_Set evaluation
count = 0
precsion = 0
recall = 0
f1socre = 0
for name in result.columns:
    #test_playlist_name 과 거리가 가까운 train_playlist_name 상위 10개 도출
    result_playlist_name = result[name].sort_values(ascending = False).head(10).index
    #print count
    
    #test_playlist 생성
    test_playlist = []
    for idx in range(0, len(test_df['name'])):
        if(test_df['name'][idx].lower() == name):
            test_track_uri = pd.DataFrame(test_df['tracks'][idx])
            test_playlist = list(test_track_uri['track_uri'])
    #print "test_playlist length :{}".format(len(test_playlist))        
    if(len(test_playlist) == 0):
        continue
            
        #비교할 train_playlist 생성
    train_playlist = []
    for train_name in result_playlist_name:
        for idx in range(0,len(train_df['name'])):
            if(train_df['name'][idx].lower() == train_name):
                df = pd.DataFrame(train_df['tracks'][idx])
                #temp =[]
                #if(len(train_playlist) >= 500):
                 #   continue
                #elif(len(train_playlist) + len(list(df['track_uri'])) >= 500):
                 #   temp = list(df['track_uri'])
                 #   train_playlist += temp[:-(len(train_playlist)-500)]
                #else:
                train_playlist += list(df['track_uri'])
    #print "total_train_playlist : {}".format(len(train_playlist))   
    #print "recall train_playlist : {}".format(len(train_playlist[:len(test_playlist)]))
    #r-precision 계산
    #if(len(set(train_playlist).intersection(set(test_playlist))) == 0):
    #    continue
    precision += 1.0 *len(set(train_playlist).intersection(set(test_playlist)))/len(set(train_playlist)) *100
    recall += 1.0 * len(set(train_playlist).
                            intersection(set(test_playlist)))/len(set(test_playlist))*100
    count+=1
    #print "r-precision summation: {}".format(c)
    #print "count : {}".format(count)
    #avarage result
precision = 1.0 * precision/(count)
recall = 1.0 * recall /(count)
f1score = (2 * precision * recall) / (precision + recall)
print "precision : {}\nrecall : {}\nf1score : {}".format(precision,recall,f1score)

precision : 0.541552472286
recall : 5.49891959348
f1score : 0.986000255718
